In [18]:
!git clone https://github.com/Vision-CAIR/LongVU
!pip install -r /content/LongVU/requirements.txt

In [2]:
!pip install huggingface-hub

In [23]:
%cd /content/LongVU

/content/LongVU
/content/LongVU
/content/LongVU


In [27]:
# LOAD MODEL

!pwd

import numpy as np
import torch
from longvu.builder import load_pretrained_model
from longvu.constants import (
    DEFAULT_IMAGE_TOKEN,
    IMAGE_TOKEN_INDEX,
)
from longvu.conversation import conv_templates, SeparatorStyle
from longvu.mm_datautils import (
    KeywordsStoppingCriteria,
    process_images,
    tokenizer_image_token,
)
from decord import cpu, VideoReader

tokenizer, model, image_processor, context_len = load_pretrained_model(
    "Vision-CAIR/LongVU_Qwen2_7B", None, "cambrian_qwen",
)

model.eval()

/content/LongVU


tokenizer_config.json:   0%|          | 0.00/1.54k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/3.51G [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/368 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/4.55G [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/436 [00:00<?, ?B/s]

CambrianQwenForCausalLM(
  (model): CambrianQwenModel(
    (embed_tokens): Embedding(151647, 3584)
    (layers): ModuleList(
      (0-27): 28 x Qwen2DecoderLayer(
        (self_attn): Qwen2SdpaAttention(
          (q_proj): Linear(in_features=3584, out_features=3584, bias=True)
          (k_proj): Linear(in_features=3584, out_features=512, bias=True)
          (v_proj): Linear(in_features=3584, out_features=512, bias=True)
          (o_proj): Linear(in_features=3584, out_features=3584, bias=False)
          (rotary_emb): Qwen2RotaryEmbedding()
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear(in_features=3584, out_features=18944, bias=False)
          (up_proj): Linear(in_features=3584, out_features=18944, bias=False)
          (down_proj): Linear(in_features=18944, out_features=3584, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm()
        (post_attention_layernorm): Qwen2RMSNorm()
      )
    )
    (norm): Qwen2RMSNorm()
    (m

In [28]:
def infer_video(video_path, qs):
  vr = VideoReader(video_path, ctx=cpu(0), num_threads=1)
  fps = float(vr.get_avg_fps())
  frame_indices = np.array([i for i in range(0, len(vr), round(fps),)])
  video = []
  for frame_index in frame_indices:
      img = vr[frame_index].asnumpy()
      video.append(img)
  video = np.stack(video)
  image_sizes = [video[0].shape[:2]]
  video = process_images(video, image_processor, model.config)
  video = [item.unsqueeze(0) for item in video]

  qs = DEFAULT_IMAGE_TOKEN + "\n" + qs
  conv = conv_templates["qwen"].copy()
  conv.append_message(conv.roles[0], qs)
  conv.append_message(conv.roles[1], None)
  prompt = conv.get_prompt()

  input_ids = tokenizer_image_token(prompt, tokenizer, IMAGE_TOKEN_INDEX, return_tensors="pt").unsqueeze(0).to(model.device)
  stop_str = conv.sep if conv.sep_style != SeparatorStyle.TWO else conv.sep2
  keywords = [stop_str]
  stopping_criteria = KeywordsStoppingCriteria(keywords, tokenizer, input_ids)

  with torch.inference_mode():
      output_ids = model.generate(
          input_ids,
          images=video,
          image_sizes=image_sizes,
          do_sample=False,
          temperature=0.2,
          max_new_tokens=20,
          use_cache=True,
          stopping_criteria=[stopping_criteria],
      )
  pred = tokenizer.batch_decode(output_ids, skip_special_tokens=True)[0].strip()

  return pred


In [29]:
from google.colab import drive

drive.mount('/content/drive')


queries_file = "/content/drive/MyDrive/data/top_50_queries/top_queries.json"
video_dir = "/content/drive/MyDrive/data/extracted_clips"
output_file = "/content/drive/MyDrive/data/predictions_full_precision_LongVU.json"
predictions = []

Mounted at /content/drive


In [30]:
# INFERENCE CODE

import json

# Caricamento delle query
with open(queries_file, 'r') as quer_file:
    queries = json.load(quer_file)

# Elaborazione delle query
for query in queries:
    prompt = query["query"]
    video_id = query["video_id"]
    ground_truth = query["ground_truth"]
    start_time, end_time = ground_truth

    # Percorso del video clip
    video_path = os.path.join(video_dir, f"{video_id}__{start_time}__{end_time}.mp4")

    if not os.path.exists(video_path):
        print(f"Video file not found: {video_path}")
        continue

    response = infer_video(video_path=video_path, qs=prompt)

    # Salvataggio della predizione
    predictions.append({
        "video_id": video_id,
        "prompt": prompt,
        "prediction": response,
    })
    print(f"Processed video {video_id}")



# Salvataggio delle predizioni in un file JSON
with open(output_file, 'w') as f:
    json.dump(predictions, f, indent=4)

print(f"Predictions saved to {output_file}")



/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:540: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.2` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token.As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Processed video 8698a0ea-f434-49f9-a8e4-45220e5d4b2c


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.


Processed video f082242c-309f-48a1-97fa-5c1d6bd255fb


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.


...

Processed video 82ace2ff-4a87-4abe-ba45-3965bbc13658


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.


Processed video 8d84ee2b-5dcb-4311-9769-a52e7b68d9e1


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.


Processed video 180b72fa-2102-414e-a6d5-2806e53f320c
Predictions saved to /content/drive/MyDrive/data/predictions_full_precision.json
